In [1]:
!pip install "flaml[automl]"
!pip install rapidfuzz


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Zhihao\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Zhihao\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import os

In [ ]:
from nq_agents import WorkflowAutogen
from nq_agents import data_extraction

# Initialize the multi-agent system
my_agents = WorkflowAutogen()

# Read input examples
GOLD_FILE = 'data/train_top100.jsonl'
num_examples = data_extraction.count_lines(GOLD_FILE)

input_examples = data_extraction.read_first_lines(GOLD_FILE, num_examples)

# Make predictions. side effect: write to file
save_path = GOLD_FILE.replace('.jsonl', '_predictions.jsonl')
predictions = my_agents.predict_batch(examples=input_examples, save_path=save_path, verbose=True)



Example 1/100
char length: 21750
token length: 3778
Generated subqueries: ['What are the primary uses of opt-in email marketing', 'What is the most common industry or sector that utilizes opt-in email marketing', 'How does opt-in email marketing differ from other types of email marketing', 'What are the benefits of using opt-in email marketing for businesses', 'Which specific company or organization has been particularly successful with opt-in email marketing', 'What role do user consent and data protection play in opt-in email marketing', 'How has the use of opt-in email marketing evolved over time']
subqueries ['What are the primary uses of opt-in email marketing', 'What is the most common industry or sector that utilizes opt-in email marketing', 'How does opt-in email marketing differ from other types of email marketing', 'What are the benefits of using opt-in email marketing for businesses', 'Which specific company or organization has been particularly successful with opt-in email

TypeError: string indices must be integers

### Evaluate


In [ ]:
from nq_agents import evaluation

# Evaluate predictions
GOLD_FILE = 'data/two-dev.jsonl'
PREDICTIONS_FILE = 'data/two-dev_predictions.jsonl' # generated by multi_agent.py

result = evaluation.evaluate_predictions(GOLD_FILE, PREDICTIONS_FILE)

In [ ]:
result

In [ ]:
a = {1:1, 2:2}
b = {1:1, 2:2}
a == b



In [ ]:
a.keys().symmetric_difference(b.keys())

In [ ]:
type(a.keys())